Praktikum NLP:

ToDo:
- Load dataset √

- Dependency Parsing

- Extracting aspects (phrases/sentiment-words)

    - Summarize aspects for each hotel

    - Calculate sentiment scores for each aspect (TextBlob, VADER, etc...)

- Analyse 5 most common aspects

- Dump into graphs for the presentation

In [17]:
#Load data

import pandas as pd
import re
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
import string
from collections import Counter
from textblob import TextBlob

#nltk.download('wordnet')
#nltk.download('omw-1.4')


with open('../../include/Hotel_Reviews.csv') as file:
#with open('Hotel_Reviews.csv') as file:
    df = pd.read_csv(file, nrows=10)


#!pip install spacy
#!python -m spacy download en_core_web_sm





nlp = spacy.load("en_core_web_sm")




In [18]:
## Clean data

# Delete redundant and empty reviews

def preprocessing_reviews(df):
    df.drop(df[ df['Negative_Review'] == 'No Negative'].index)
    df.drop(df[ df['Positive_Review'] == 'No Positive'].index)
    df.drop_duplicates(keep=False)

# Split into sentences and add dots to positive reviews and negative reviews
    for text in df['Positive_Review']:
        parts_of_review = re.findall('[A-Z][^A-Z]*', text)
        for index, sentence in enumerate(parts_of_review):
            if "I " or "i " in sentence:
                parts_of_review[index : index +1] = [''.join(parts_of_review[index : index + 1])]
            parts_of_review[index] = sentence.strip() + "."
        complete_review = ' '.join(parts_of_review)

        # correct 'n t'|'nt'|'n t' to ' not'
        complete_review = re.sub(r'n\s+t |n\'t ', ' not',complete_review)
        df['Positive_Review'] = df['Positive_Review'].replace([text], complete_review)

    for text in df['Negative_Review']:
        parts_of_review = re.findall('[A-Z][^A-Z]*', text)
        for index, sentence in enumerate(parts_of_review):
            if "I " or "i " in sentence:
                parts_of_review[index : index +1] = [''.join(parts_of_review[index : index + 1])]
            parts_of_review[index] = sentence.strip() + "."
        complete_review = ' '.join(parts_of_review)

        # correct 'n t'|'nt'|'n t' to ' not'
        complete_review = re.sub(r'n\s+t |n\'t ', ' not',complete_review)
        #        complete_review = re.sub(r'n\s+t|n\'t|nt ', ' not',complete_review)
        df['Negative_Review'] = df['Negative_Review'].replace([text], complete_review)

    return df


df = preprocessing_reviews(df)
pd.set_option("display.max_colwidth", None) # -1?

#TODO: Maybe i and I will lead to problems...
#TODO: payments <= payme nots, restaurants <= restaura not, romantic <= Roma notic
# My room was dirty and. I was afraid to walk barefoot o nothe floor 

# Quick check if wanted
#print(df['Negative_Review'])
#print(df['Positive_Review'])




# Remove stopwords

#nlp = spacy.load("en_core_web_sm")
#stopword_list = nltk.corpus.stopwords.words('english')


# lower case


# Lemmatization

print(df)




                                               Hotel_Address  \
0   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
1   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
2   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
3   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
4   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
5   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
6   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
7   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
8   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   
9   s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands   

   Additional_Number_of_Scoring Review_Date  Average_Score   Hotel_Name  \
0                           194    8/3/2017            7.7  Hotel Arena   
1                           194    8/3/2017            7.7  Hotel Arena   
2                           194   7/31/2017            7.7  Hotel Aren

In [19]:
# maybe check for words with "...nt" and then split the nt. Then check, if it's a verb. If yes, change from "didnt" to "did not" with a regex. (If not, leave it)

aspects_pos = []
aspects_neg = []


for review in df["Negative_Review"]:
  doc = nlp(review)
  descriptors = ''
  aspect = ''
  for token in doc:
    negation = False
    for negative_token in doc:
      if negative_token.dep_ == 'neg' and negative_token.head == token: negation = True 
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      aspect = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV': 
          continue
        prepend += child.text + ' '
      descriptors = prepend + token.text
  aspects_neg.append({'aspect': aspect,
    'description': descriptors, 'negation': negation})
aspects_neg = pd.DataFrame(aspects_neg)
#print(pd.DataFrame(aspects_neg))

  #negation.extend([tok for tok in doc if tok.dep_ == 'neg'])
for review in df["Positive_Review"]:
  doc = nlp(review)
  descriptors = ''
  aspect = ''
  for token in doc:
    negation = False
    for negative_token in doc:
      if negative_token.dep_ == 'neg' and negative_token.head == token: negation = True 
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      aspect = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptors = prepend + token.text
  aspects_pos.append({'aspect': aspect,
    'description': descriptors, 'negation': negation})
aspects_pos = pd.DataFrame(aspects_pos)
#print(pd.DataFrame(aspects_pos))





In [20]:

for boolean in aspects_neg['negation']:
    if boolean == True: print("true")

for boolean in aspects_pos['negation']:
    if boolean == True: print("true")


In [21]:
"""countStemmer = Counter()

for aspect in aspects['aspect']:
    aspect = stemmer.stem( aspect)
    aspect = aspect.lower()
    countStemmer[aspect] += 1

print(countStemmer)"""


lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

countLemmaPos = Counter()
countLemmaNeg = Counter()


for aspect in aspects_pos['aspect']:
    aspect = lemmatizer.lemmatize(aspect)
    aspect = aspect.lower()
    countLemmaPos[aspect] += 1


for aspect in aspects_neg['aspect']:
    aspect = lemmatizer.lemmatize(aspect)
    aspect = aspect.lower()
    countLemmaNeg[aspect] += 1

print(countLemmaNeg, "\n")
print(countLemmaPos)



    

Counter({'room': 3, '': 3, 'offer': 1, 'photo': 1, 'mess': 1, 'noise': 1}) 

Counter({'': 4, 'park': 1, 'option': 1, 'range': 1, 'building': 1, 'room': 1, 'people': 1})


In [22]:
sentiment_pos = []
for aspect in aspects_pos['description']:
  sentiment_pos.append(TextBlob(aspect).sentiment)
aspects_pos['sentiment'] = sentiment_pos

sentiment_neg = []
for aspect in aspects_neg['description']:
  sentiment_neg.append(TextBlob(aspect).sentiment)
aspects_neg['sentiment'] = sentiment_neg


In [23]:
print(aspects_neg)

   aspect description  negation                                 sentiment
0    room       newer     False                                (0.0, 0.0)
1            Negative     False                               (-0.3, 0.4)
2   rooms       empty     False                               (-0.1, 0.5)
3   offer       ready     False                                (0.2, 0.5)
4  photos       happy     False                                (0.8, 1.0)
5    mess       total     False                               (0.0, 0.75)
6                         False                                (0.0, 0.0)
7           very good     False  (0.9099999999999999, 0.7800000000000001)
8    room       ready     False                                (0.2, 0.5)
9   noise     so much     False                                (0.2, 0.2)


In [24]:
# Dependency parsing

import nltk



def dependency_parsing(df):
    for sentence in df:
        break
  #      doc = nlp(sentence)




In [25]:
# In Addition:

# I assume we are not supposed to do topic modeling, just "simple" rule based aspect extraction... right?

# make Sentiment Analysis (only simple one: Textblob) for positive and negative reviews and check, if it fits,
#                otherwise drop the review
# -> example: "I am so angry that i made this post available via all possible sites i use when planing my trips so
#              no one will make the mistake of booking this place"
# maybe handle orthographical mistakes before splitting into sentences
#



In [26]:
"""countStemmer = Counter()

for aspect in aspects['aspect']:
    aspect = stemmer.stem( aspect)
    aspect = aspect.lower()
    countStemmer[aspect] += 1

print(countStemmer)"""


lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

countLemmaPos = Counter()
countLemmaNeg = Counter()


for aspect in aspects_pos['aspect']:
    aspect = lemmatizer.lemmatize(aspect)
    aspect = aspect.lower()
    countLemmaPos[aspect] += 1


for aspect in aspects_neg['aspect']:
    aspect = lemmatizer.lemmatize(aspect)
    aspect = aspect.lower()
    countLemmaNeg[aspect] += 1

print(countLemmaNeg, "\n")
print(countLemmaPos)



    

Counter({'room': 3, '': 3, 'offer': 1, 'photo': 1, 'mess': 1, 'noise': 1}) 

Counter({'': 4, 'park': 1, 'option': 1, 'range': 1, 'building': 1, 'room': 1, 'people': 1})


In [27]:
sentiment_pos = []
for aspect in aspects_pos['description']:
  sentiment_pos.append(TextBlob(aspect).sentiment)
aspects_pos['sentiment'] = sentiment_pos

sentiment_neg = []
for aspect in aspects_neg['description']:
  sentiment_neg.append(TextBlob(aspect).sentiment)
aspects_neg['sentiment'] = sentiment_neg


In [28]:
print(aspects_neg)

   aspect description  negation                                 sentiment
0    room       newer     False                                (0.0, 0.0)
1            Negative     False                               (-0.3, 0.4)
2   rooms       empty     False                               (-0.1, 0.5)
3   offer       ready     False                                (0.2, 0.5)
4  photos       happy     False                                (0.8, 1.0)
5    mess       total     False                               (0.0, 0.75)
6                         False                                (0.0, 0.0)
7           very good     False  (0.9099999999999999, 0.7800000000000001)
8    room       ready     False                                (0.2, 0.5)
9   noise     so much     False                                (0.2, 0.2)


In [29]:
# Dependency parsing

import nltk



def dependency_parsing(df):
    for sentence in df:
        break
  #      doc = nlp(sentence)




In [30]:
# In Addition:

# I assume we are not supposed to do topic modeling, just "simple" rule based aspect extraction... right?

# make Sentiment Analysis (only simple one: Textblob) for positive and negative reviews and check, if it fits,
#                otherwise drop the review
# -> example: "I am so angry that i made this post available via all possible sites i use when planing my trips so
#              no one will make the mistake of booking this place"
# maybe handle orthographical mistakes before splitting into sentences
#



In [31]:
# Define rules for extracting aspects: keywords:

# Aspect food: restaurant, breakfast, dinner, lunch, bar
# Aspect helpfulness: staff, personell, clerk, service, check-in
# Aspect price: cheap, expensive, euros, ...?
# Aspect cleanliness: bed, matress, bathroom, floor, sheets, "changed daily", shower
# Aspect features/equipment:  USB-port, wifi, airconditioning, showergel, shampoo, furniture, roomservice, bathtub
# Aspect connectivity: bus, tram, train, airportshuttle, parking
# Aspect property: stairs (steep), building, (under-)construction, renovation, location, citycenter, sightseeing


Presentation: Spatial-Mapping where hotels are // where guests are from
=> maybe what aspects are mentioned by primarily by guests by nationality or positive/negative in general

